# Using Neural Networks

In [3]:
import tensorflow
import xgboost
import sklearn

print('The xgboost version is {}.'.format(xgboost.__version__))
print('The scikit-learn version is {}.'.format(sklearn.__version__))
print('The tensorflow version is {}.'.format(tensorflow.__version__))


# The nltk version is 3.0.0.
# The scikit-learn version is 0.15.2.

The xgboost version is 1.2.1.
The scikit-learn version is 0.23.2.
The tensorflow version is 2.3.1.


## Define evaluation matrices

In [4]:
%%time
#==============================================================================
# Defining the function to vaiidate the model with the test data and
# get the results from regression evaluation metrices in sklearn
#==============================================================================
pred = []
accuracy = []


def test_data_regression(y_pred, y_test,  model='NN', comment1= 'NA', comment2 = '', comment3 = ''):
    pred = []
    accuracy = []

    #==============================================================================
    # Compute performance
    #==============================================================================

    from sklearn.metrics import mean_absolute_error
    mae = mean_absolute_error(y_test, y_pred)
    print(mae, ' mean_absolute_error')
    accuracy.append(mae)

    from sklearn.metrics import mean_squared_error
    mse = mean_squared_error(y_test, y_pred, squared=True)
    print(mse, ' mean_squared_error')
    accuracy.append(mse)

    from sklearn.metrics import mean_squared_error
    rmse = mean_squared_error(y_test, y_pred, squared=False)
    print(rmse, ' root_mean_squared_error')
    accuracy.append(rmse)

    from sklearn.metrics import r2_score
    r2 = r2_score(y_test, y_pred)
    print(r2, ' r2_score')
    accuracy.append(r2)

    # Print accuracy computed from predictions on the test set
    print(accuracy)

    #==============================================================================
    # Append Results
    #==============================================================================
    results = []
    import datetime
    datetime = datetime.datetime.now()
    results.append((model, 'MAE = {}'.format(mae), 'MSE = {}'.format(mse),
                    'RMSE = {}'.format(rmse), 'R2 = {}'.format(r2),
                    'List = {}'.format(accuracy), datetime, '', comment1, comment2, comment3))

    pd.DataFrame(np.asarray(results)).to_csv('results.csv',
                                             mode='a',
                                             header=None)
    pred.extend(y_pred)

Wall time: 0 ns


## Import data

In [5]:
import pandas as pd

ml_df = pd.read_csv("ml_df.csv")
ml_df

,blue_median,green_median,red_median,red_edge_median,ndvi_median,nir_median,mtci_values,evi_values,age,grain_yield
0,0.034152,0.085196,0.045748,0.174878,0.883373,0.734195,4.331436,0.982070,48,431.698672
1,0.036060,0.084995,0.045884,0.168578,0.876906,0.692295,4.268514,0.952200,48,409.089032
2,0.036943,0.089985,0.045336,0.180775,0.888153,0.759447,4.272559,1.017603,48,372.860721
3,0.030649,0.072090,0.036053,0.151221,0.906785,0.739433,5.107437,1.018870,48,528.219355
4,0.037586,0.085404,0.050601,0.167592,0.869756,0.713398,4.665330,0.954982,48,462.505958
...,...,...,...,...,...,...,...,...,...,...
9265,0.029813,0.115001,0.043041,0.235598,0.896805,0.778433,3.080369,1.906517,51,631.004782
9266,0.028875,0.116390,0.041759,0.235039,0.903283,0.792686,2.926750,1.949479,51,641.486148
9267,0.028778,0.119711,0.039235,0.255132,0.916507,0.884904,2.912663,2.181455,51,562.676888
9268,0.026632,0.106301,0.036410,0.221450,0.926392,0.858537,3.239740,2.120530,51,547.551879


In [6]:
def get_data():
    #get train data
    train_data_path ='ml_df.csv'
    data = pd.read_csv(train_data_path)
    
    return data

def get_input_target_data():
  #reading train data
  data = get_data()

  target_all = data.grain_yield
  data.drop(['grain_yield'],axis = 1 , inplace = True)


  return data, target_all

#Load train and test data into pandas DataFrames
data, target_all = get_input_target_data()


In [7]:
target_all.shape

(9270,)

## Test train split


In [8]:
%%time

from sklearn.model_selection import train_test_split

#==============================================================================
# Create separate train/test splits from Main data
#==============================================================================
train, test, target, val = train_test_split(data,
                                            target_all,
                                            test_size=0.3,
                                            shuffle = True,
                                            random_state=55)

Wall time: 2.99 ms


In [9]:
%%time
from sklearn.preprocessing import StandardScaler
#==============================================================================
# Scale features using StandardScaler class in scikit-learn
#==============================================================================

# Initialise standard scaler and compute mean and STD from training data
sc = StandardScaler()
sc.fit(train)

# Transform (standardise) both X_train and X_test with mean and STD from
# training data
train = sc.transform(train)
test = sc.transform(test)

Wall time: 6.98 ms


## Import libraries

In [10]:
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error 
from matplotlib import pyplot as plt
import seaborn as sb
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import warnings 
warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', category=DeprecationWarning)
from xgboost import XGBRegressor

## Make the Deep Neural Network

In [11]:
%%time
NN_model = Sequential()

Wall time: 22.7 ms


## The Input Layer

In [12]:
%%time
NN_model.add(Dense(128, kernel_initializer='normal',input_dim = train.shape[1], activation='relu'))

Wall time: 16 ms


## The Hidden Layers

In [13]:
%%time
NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))

Wall time: 44.5 ms


## The Output Layer

In [14]:
%%time
NN_model.add(Dense(1, kernel_initializer='normal',activation='linear'))

Wall time: 5.19 ms


## Compile the network

In [15]:
%%time
NN_model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])
NN_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 128)               1280      
_________________________________________________________________
dense_1 (Dense)              (None, 256)               33024     
_________________________________________________________________
dense_2 (Dense)              (None, 256)               65792     
_________________________________________________________________
dense_3 (Dense)              (None, 256)               65792     
_________________________________________________________________
dense_4 (Dense)              (None, 256)               65792     
_________________________________________________________________
dense_5 (Dense)              (None, 256)               65792     
_________________________________________________________________
dense_6 (Dense)              (None, 256)               6

## Define a checkpoint callback :

In [16]:
%%time
checkpoint_name = 'nn_checkpoint/Weights-{epoch:03d}--{val_loss:.5f}.hdf5' 
checkpoint = ModelCheckpoint(checkpoint_name, monitor='val_loss', verbose = 1, save_best_only = True, mode ='auto')
callbacks_list = [checkpoint]

Wall time: 0 ns


## Third : Train the model :

In [18]:
%%time
NN_model.fit(train, target, epochs=500, batch_size=32, validation_split = 0.2, callbacks=callbacks_list)

Epoch 1/500
156/163 [===========================>..] - ETA: 0s - loss: 137.9672 - mean_absolute_error: 137.9672- ETA: 0s - loss: 197.0086 - mean_absolute_error: 197.
Epoch 00001: val_loss improved from inf to 100.54611, saving model to nn_checkpoint\Weights-001--100.54611.hdf5
163/163 [==============================] - 1s 5ms/step - loss: 135.0129 - mean_absolute_error: 135.0129 - val_loss: 100.5461 - val_mean_absolute_error: 100.5461
Epoch 2/500
153/163 [===========================>..] - ETA: 0s - loss: 56.6291 - mean_absolute_error: 56.6291
Epoch 00002: val_loss improved from 100.54611 to 47.41439, saving model to nn_checkpoint\Weights-002--47.41439.hdf5
163/163 [==============================] - 1s 4ms/step - loss: 56.2968 - mean_absolute_error: 56.2968 - val_loss: 47.4144 - val_mean_absolute_error: 47.4144
Epoch 3/500
153/163 [===========================>..] - ETA: 0s - loss: 53.9592 - mean_absolute_error: 53.9592- ETA: 0s - loss: 53.9974 - mean_absolute_error: 53.99
Epoch 00003: v

Epoch 00024: val_loss did not improve from 40.69478
163/163 [==============================] - 1s 3ms/step - loss: 44.0690 - mean_absolute_error: 44.0690 - val_loss: 41.1440 - val_mean_absolute_error: 41.1440
Epoch 25/500
153/163 [===========================>..] - ETA: 0s - loss: 43.2983 - mean_absolute_error: 43.2983
Epoch 00025: val_loss did not improve from 40.69478
163/163 [==============================] - 1s 3ms/step - loss: 43.3140 - mean_absolute_error: 43.3140 - val_loss: 46.5439 - val_mean_absolute_error: 46.5439
Epoch 26/500
161/163 [============================>.] - ETA: 0s - loss: 42.9121 - mean_absolute_error: 42.9121
Epoch 00026: val_loss did not improve from 40.69478
163/163 [==============================] - 1s 3ms/step - loss: 42.8788 - mean_absolute_error: 42.8788 - val_loss: 43.5302 - val_mean_absolute_error: 43.5302
Epoch 27/500
153/163 [===========================>..] - ETA: 0s - loss: 41.9946 - mean_absolute_error: 41.9946
Epoch 00027: val_loss did not improve fr

Epoch 48/500
147/163 [==========================>...] - ETA: 0s - loss: 43.0535 - mean_absolute_error: 43.0535
Epoch 00048: val_loss did not improve from 37.76737
163/163 [==============================] - 1s 3ms/step - loss: 42.6506 - mean_absolute_error: 42.6506 - val_loss: 39.0499 - val_mean_absolute_error: 39.0499
Epoch 49/500
151/163 [==========================>...] - ETA: 0s - loss: 41.0697 - mean_absolute_error: 41.0697
Epoch 00049: val_loss did not improve from 37.76737
163/163 [==============================] - 1s 3ms/step - loss: 41.2830 - mean_absolute_error: 41.2830 - val_loss: 39.5492 - val_mean_absolute_error: 39.5492
Epoch 50/500
150/163 [==========================>...] - ETA: 0s - loss: 41.0766 - mean_absolute_error: 41.0766- ETA: 0s - loss: 42.6845 - mean_absolute_error:  - ETA: 0s - loss: 41.1677 - mean_absolute_error: 41.167
Epoch 00050: val_loss did not improve from 37.76737
163/163 [==============================] - 1s 3ms/step - loss: 41.1520 - mean_absolute_error

157/163 [===========================>..] - ETA: 0s - loss: 39.5063 - mean_absolute_error: 39.5063
Epoch 00073: val_loss did not improve from 37.76737
163/163 [==============================] - 1s 3ms/step - loss: 39.6120 - mean_absolute_error: 39.6120 - val_loss: 41.0918 - val_mean_absolute_error: 41.0918
Epoch 74/500
147/163 [==========================>...] - ETA: 0s - loss: 38.6369 - mean_absolute_error: 38.6369
Epoch 00074: val_loss did not improve from 37.76737
163/163 [==============================] - 1s 3ms/step - loss: 39.0492 - mean_absolute_error: 39.0492 - val_loss: 44.9106 - val_mean_absolute_error: 44.9106
Epoch 75/500
160/163 [============================>.] - ETA: 0s - loss: 39.2230 - mean_absolute_error: 39.2230
Epoch 00075: val_loss did not improve from 37.76737
163/163 [==============================] - 1s 4ms/step - loss: 39.2465 - mean_absolute_error: 39.2465 - val_loss: 40.5364 - val_mean_absolute_error: 40.5364
Epoch 76/500
159/163 [============================>.]

Epoch 99/500
161/163 [============================>.] - ETA: 0s - loss: 38.4180 - mean_absolute_error: 38.4180
Epoch 00099: val_loss did not improve from 37.52567
163/163 [==============================] - 1s 4ms/step - loss: 38.3906 - mean_absolute_error: 38.3906 - val_loss: 47.7685 - val_mean_absolute_error: 47.7685
Epoch 100/500
157/163 [===========================>..] - ETA: 0s - loss: 38.0259 - mean_absolute_error: 38.0259
Epoch 00100: val_loss did not improve from 37.52567
163/163 [==============================] - 1s 4ms/step - loss: 37.9655 - mean_absolute_error: 37.9655 - val_loss: 39.8583 - val_mean_absolute_error: 39.8583
Epoch 101/500
158/163 [============================>.] - ETA: 0s - loss: 37.7922 - mean_absolute_error: 37.7922
Epoch 00101: val_loss did not improve from 37.52567
163/163 [==============================] - 1s 3ms/step - loss: 37.7877 - mean_absolute_error: 37.7877 - val_loss: 37.5298 - val_mean_absolute_error: 37.5298
Epoch 102/500
160/163 [===============

...

Epoch 382/500
161/163 [============================>.] - ETA: 0s - loss: 31.5130 - mean_absolute_error: 31.5130
Epoch 00382: val_loss did not improve from 37.36064
163/163 [==============================] - 0s 3ms/step - loss: 31.4603 - mean_absolute_error: 31.4603 - val_loss: 41.0049 - val_mean_absolute_error: 41.0049
Epoch 383/500
163/163 [==============================] - ETA: 0s - loss: 31.7270 - mean_absolute_error: 31.7270
Epoch 00383: val_loss did not improve from 37.36064
163/163 [==============================] - 0s 3ms/step - loss: 31.7270 - mean_absolute_error: 31.7270 - val_loss: 39.2527 - val_mean_absolute_error: 39.2527
Epoch 384/500
156/163 [===========================>..] - ETA: 0s - loss: 31.6199 - mean_absolute_error: 31.6199
Epoch 00384: val_loss did not improve from 37.36064
163/163 [==============================] - 0s 3ms/step - loss: 31.4670 - mean_absolute_error: 31.4670 - val_loss: 38.2911 - val_mean_absolute_error: 38.2911
Epoch 385/500
151/163 [==============

Epoch 408/500
158/163 [============================>.] - ETA: 0s - loss: 31.7166 - mean_absolute_error: 31.7166
Epoch 00408: val_loss did not improve from 37.36064
163/163 [==============================] - 0s 3ms/step - loss: 31.7840 - mean_absolute_error: 31.7840 - val_loss: 44.3785 - val_mean_absolute_error: 44.3785
Epoch 409/500
161/163 [============================>.] - ETA: 0s - loss: 31.9505 - mean_absolute_error: 31.9505
Epoch 00409: val_loss did not improve from 37.36064
163/163 [==============================] - 0s 3ms/step - loss: 31.9458 - mean_absolute_error: 31.9458 - val_loss: 39.0547 - val_mean_absolute_error: 39.0547
Epoch 410/500
163/163 [==============================] - ETA: 0s - loss: 31.7830 - mean_absolute_error: 31.7830
Epoch 00410: val_loss did not improve from 37.36064
163/163 [==============================] - 0s 3ms/step - loss: 31.7830 - mean_absolute_error: 31.7830 - val_loss: 40.7254 - val_mean_absolute_error: 40.7254
Epoch 411/500
159/163 [==============

Epoch 434/500
152/163 [==========================>...] - ETA: 0s - loss: 30.8880 - mean_absolute_error: 30.8880
Epoch 00434: val_loss did not improve from 37.36064
163/163 [==============================] - 0s 3ms/step - loss: 31.1482 - mean_absolute_error: 31.1482 - val_loss: 39.3064 - val_mean_absolute_error: 39.3064
Epoch 435/500
150/163 [==========================>...] - ETA: 0s - loss: 30.7111 - mean_absolute_error: 30.7111
Epoch 00435: val_loss did not improve from 37.36064
163/163 [==============================] - 0s 3ms/step - loss: 30.5465 - mean_absolute_error: 30.5465 - val_loss: 39.1866 - val_mean_absolute_error: 39.1866
Epoch 436/500
154/163 [===========================>..] - ETA: 0s - loss: 30.6951 - mean_absolute_error: 30.6951
Epoch 00436: val_loss did not improve from 37.36064
163/163 [==============================] - 0s 3ms/step - loss: 30.7281 - mean_absolute_error: 30.7281 - val_loss: 38.7654 - val_mean_absolute_error: 38.7654
Epoch 437/500
151/163 [==============

Epoch 460/500
148/163 [==========================>...] - ETA: 0s - loss: 30.3037 - mean_absolute_error: 30.3037
Epoch 00460: val_loss did not improve from 37.36064
163/163 [==============================] - 1s 3ms/step - loss: 30.3442 - mean_absolute_error: 30.3442 - val_loss: 39.9906 - val_mean_absolute_error: 39.9906
Epoch 461/500
151/163 [==========================>...] - ETA: 0s - loss: 30.8978 - mean_absolute_error: 30.8978
Epoch 00461: val_loss did not improve from 37.36064
163/163 [==============================] - 0s 3ms/step - loss: 30.8644 - mean_absolute_error: 30.8644 - val_loss: 40.7149 - val_mean_absolute_error: 40.7149
Epoch 462/500
151/163 [==========================>...] - ETA: 0s - loss: 29.8738 - mean_absolute_error: 29.8738
Epoch 00462: val_loss did not improve from 37.36064
163/163 [==============================] - 0s 3ms/step - loss: 30.1607 - mean_absolute_error: 30.1607 - val_loss: 39.6227 - val_mean_absolute_error: 39.6227
Epoch 463/500
149/163 [==============

Epoch 486/500
151/163 [==========================>...] - ETA: 0s - loss: 30.3919 - mean_absolute_error: 30.3919
Epoch 00486: val_loss did not improve from 37.36064
163/163 [==============================] - 1s 3ms/step - loss: 30.5360 - mean_absolute_error: 30.5360 - val_loss: 43.0696 - val_mean_absolute_error: 43.0696
Epoch 487/500
153/163 [===========================>..] - ETA: 0s - loss: 30.0620 - mean_absolute_error: 30.0620
Epoch 00487: val_loss did not improve from 37.36064
163/163 [==============================] - 1s 3ms/step - loss: 30.3136 - mean_absolute_error: 30.3136 - val_loss: 39.5224 - val_mean_absolute_error: 39.5224
Epoch 488/500
144/163 [=========================>....] - ETA: 0s - loss: 30.4717 - mean_absolute_error: 30.4717
Epoch 00488: val_loss did not improve from 37.36064
163/163 [==============================] - 0s 3ms/step - loss: 30.2118 - mean_absolute_error: 30.2118 - val_loss: 39.0435 - val_mean_absolute_error: 39.0435
Epoch 489/500
147/163 [==============

In [19]:
%%time
# Load wights file of the best model :
wights_file = 'nn_checkpoint/Weights-233--37.36064.hdf5' # choose the best checkpoint 
NN_model.load_weights(wights_file) # load it
NN_model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])

Wall time: 25.7 ms


We see that the validation loss of the best model is 18738.19

 ## Fourth : Test the model

In [20]:
%%time
predictions = NN_model.predict(test)

Wall time: 232 ms


In [21]:
%%time
diff=[]
for i in range(len(predictions)):
    diff_temp = predictions[i] -val.values[i]
    diff.append(diff_temp)
#     print(diff_temp)
print(min(diff), max(diff))


std = np.std(diff)
print(std)

[-190.10223] [363.51886]
50.60558
Wall time: 20 ms


In [23]:
%%time
test_data_regression(predictions, val, '', 'Shuffled test train split', 'Standardized data')


38.270097912041734  mean_absolute_error
2570.249690178348  mean_squared_error
50.69763002526201  root_mean_squared_error
0.833658735255884  r2_score
[38.270097912041734, 2570.249690178348, 50.69763002526201, 0.833658735255884]
Wall time: 3.97 ms


In [52]:
%%time
test_data_regression(predictions, val, '', 'Shuffled test train split')


40.268938468954225  mean_absolute_error
2992.4329133723945  mean_squared_error
54.70313440171773  root_mean_squared_error
0.8063359068286691  r2_score
[40.268938468954225, 2992.4329133723945, 54.70313440171773, 0.8063359068286691]
Wall time: 6.39 ms


Not bad at all, with some more preprocessing, and more training, we can do better.

# Using sklearn regressors

In [24]:
%%time
from sklearn.model_selection import train_test_split

# train_X, val_X, train_y, val_y = train_test_split(data, target_all, test_size = 0.25, shuffle= True,random_state = 14)


train_X = train
val_X = test
train_y = target
val_y = val

Wall time: 0 ns


In [25]:
%%time

from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor()
model.fit(train_X,train_y)

Wall time: 2.85 s


RandomForestRegressor()

In [26]:
%%time
from sklearn.metrics import mean_absolute_error

predicted_prices = model.predict(val_X)
MAE = mean_absolute_error(val_y , predicted_prices)
print('Random forest validation MAE = ', MAE)

Random forest validation MAE =  38.536555504345785
Wall time: 54.3 ms


In [29]:
%%time
test_data_regression(predicted_prices, val_y, model, 'Random Forest', 'age included', 'StandardScaler, shuffle')


38.536555504345785  mean_absolute_error
2744.0147602402244  mean_squared_error
52.383344301793336  root_mean_squared_error
0.8224130179106415  r2_score
[38.536555504345785, 2744.0147602402244, 52.383344301793336, 0.8224130179106415]
Wall time: 4.99 ms


In [16]:
%%time
test_data_regression(predicted_prices, val_y, model, 'Random Forest', 'age included', 'shuffle')


37.031414405305114  mean_absolute_error
2454.5334610906866  mean_squared_error
49.543248390579784  root_mean_squared_error
0.8387331101268987  r2_score
[37.031414405305114, 2454.5334610906866, 49.543248390579784, 0.8387331101268987]
Wall time: 7.38 ms


In [19]:
# import sklearn
# sorted(sklearn.metrics.SCORERS.keys())

## Fine tuning using grid search

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV


# Define the pipeline
pipe_line = make_pipeline(RandomForestRegressor())

# Define ranges of parameter values:
param_range  = list(range(1,200))                   # For n_components
param_range2 = list(range(10, 100, 10))          # For max_iter
param_range3 = list(range(1, 25,3))                   # For max_depth
param_range4 = [x/10 for x in list(range(0, 10))]   # For learning_rate
param_range5  = list(range(5,20))                   # For n_components


# estimator.get_params().keys()
# pipe_line.get_params().keys()

gs = GridSearchCV(estimator=pipe_line, 
                  param_grid=[{'randomforestregressor__n_estimators': param_range2,
                              'randomforestregressor__max_depth': param_range3,
                               'randomforestregressor__min_samples_split': param_range3}], 
                  scoring='neg_root_mean_squared_error', 
                  cv=3,
                  n_jobs=-1)

gs = gs.fit(X, y)
print(gs.best_score_)
print(gs.best_params_)


# Inspect AUC of parameter grid combinations
for r, _ in enumerate(grid.cv_results_['mean_test_score']):
    print("%0.3f +/- %0.2f %r"
          % (grid.cv_results_['mean_test_score'][r], 
             grid.cv_results_['std_test_score'][r] / 2.0, 
             grid.cv_results_['params'][r]))

# Using XGBOOST

In [30]:
%%time
from xgboost import XGBRegressor
XGBModel = XGBRegressor()
XGBModel.fit(train_X,train_y , verbose=False)

Wall time: 339 ms


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.300000012, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=100, n_jobs=0, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [31]:
%%time
XGBpredictions = XGBModel.predict(val_X)
MAE = mean_absolute_error(val_y , XGBpredictions)
print('XGBoost validation MAE = ',MAE)

XGBoost validation MAE =  40.40873643636777
Wall time: 8.35 ms


In [32]:
%%time

test_data_regression(XGBpredictions, val_y, XGBModel, 'XGBpredictions', 'age included', 'StandardScaler shuffle')


40.40873643636777  mean_absolute_error
3018.9793536110706  mean_squared_error
54.94523959007796  root_mean_squared_error
0.8046178759071483  r2_score
[40.40873643636777, 3018.9793536110706, 54.94523959007796, 0.8046178759071483]
Wall time: 5.98 ms


In [25]:
%%time

test_data_regression(XGBpredictions, val_y, XGBModel, 'XGBpredictions', 'age included')


38.84113664369768  mean_absolute_error
2717.9945151945826  mean_squared_error
52.13438898840747  root_mean_squared_error
0.8214232850739762  r2_score
[38.84113664369768, 2717.9945151945826, 52.13438898840747, 0.8214232850739762]
Wall time: 11.3 ms


# Using Catboost

In [85]:
%%time
from catboost import CatBoostRegressor
CatBoostModel = CatBoostRegressor()


CatBoostModel.fit(train_X,train_y , verbose=False)

Wall time: 4.9 s


In [86]:
%%time
CatBoostpredictions = CatBoostModel.predict(val_X)
MAE = mean_absolute_error(val_y , CatBoostpredictions)
print('XGBoost validation MAE = ',MAE)

XGBoost validation MAE =  37.273405610893555
Wall time: 6.95 ms


In [87]:
%%time

test_data_regression(CatBoostpredictions, val_y, CatBoostModel, 'CatBoostpredictions', 'age included')


37.273405610893555  mean_absolute_error
2443.5808884591224  mean_squared_error
49.432589335974725  root_mean_squared_error
0.8394527121826054  r2_score
[37.273405610893555, 2443.5808884591224, 49.432589335974725, 0.8394527121826054]
Wall time: 7.33 ms


In [82]:
from sklearn.model_selection import GridSearchCV

    
parameters = {'depth': [4,5,6,7,8,9, 10],
              'learning_rate' : [0.01,0.02,0.03],
              'iterations': [10, 20,30,40,50,60,70,80,90, 100]}


In [83]:
CatBoostModel = CatBoostRegressor()

Grid_CBR = GridSearchCV(estimator=CatBoostModel, scoring = 'r2', param_grid = parameters, cv = 2, n_jobs=-1)
Grid_CBR.fit(data, target_all)

0:	learn: 121.0700720	total: 3.04ms	remaining: 240ms
1:	learn: 119.1324819	total: 5.42ms	remaining: 211ms
2:	learn: 117.1401434	total: 7.96ms	remaining: 204ms
3:	learn: 115.3778416	total: 10.3ms	remaining: 196ms
4:	learn: 113.6808977	total: 12.5ms	remaining: 188ms
5:	learn: 112.0952054	total: 15ms	remaining: 185ms
6:	learn: 110.3419524	total: 17.4ms	remaining: 182ms
7:	learn: 108.7709126	total: 19.9ms	remaining: 179ms
8:	learn: 107.1308329	total: 22.5ms	remaining: 178ms
9:	learn: 105.5486467	total: 25ms	remaining: 175ms
10:	learn: 104.0190206	total: 29.1ms	remaining: 183ms
11:	learn: 102.6462254	total: 31.8ms	remaining: 180ms
12:	learn: 101.3956343	total: 34.6ms	remaining: 178ms
13:	learn: 100.2519722	total: 37.3ms	remaining: 176ms
14:	learn: 99.0404708	total: 40ms	remaining: 173ms
15:	learn: 97.7398832	total: 42.7ms	remaining: 171ms
16:	learn: 96.5827274	total: 45.2ms	remaining: 168ms
17:	learn: 95.4439522	total: 47.8ms	remaining: 165ms
18:	learn: 94.3977064	total: 50.2ms	remaining: 1

GridSearchCV(cv=2,
             estimator=<catboost.core.CatBoostRegressor object at 0x000001D8299CE5E0>,
             n_jobs=-1,
             param_grid={'depth': [4, 5, 6, 7, 8, 9, 10],
                         'iterations': [10, 20, 30, 40, 50, 60, 70, 80, 90,
                                        100],
                         'learning_rate': [0.01, 0.02, 0.03]},
             scoring='r2')

In [84]:
print(" Results from Grid Search " )
print("\n The best estimator across ALL searched params:\n",Grid_CBR.best_estimator_)
print("\n The best score across ALL searched params:\n",Grid_CBR.best_score_)
print("\n The best parameters across ALL searched params:\n",Grid_CBR.best_params_)


 Results from Grid Search 

 The best estimator across ALL searched params:

 The best score across ALL searched params:
 -1.0738438391806138

 The best parameters across ALL searched params:
 {'depth': 4, 'iterations': 80, 'learning_rate': 0.03}


In [ ]:
from sklearn.cross_decomposition import PLSRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
import time

pipe_svc = PLSRegression()

param_grid = {'max_iter':[1,2, 3, 4, 5, 6, 7, 8, 9],
              'scale' : [True]
             }

gs = GridSearchCV(estimator  = pipe_svc,
                  param_grid = param_grid,
                  scoring    = 'r2',
                  cv         = 3,
                  n_jobs=-1)

gs.fit(data,target_all)
print(gs.best_score_)
print(gs.best_params_)

In [ ]:
from sklearn.svm import SVR
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
import time

pipe_svc = SVR()

param_grid = {'C':[1,10,100,1000],'gamma':[1,0.1,0.001,0.0001], 'kernel':['linear','rbf']}

gs = GridSearchCV(estimator  = pipe_svc,
                  param_grid = param_grid,
                  scoring    = 'r2',
                  cv         = 3,
                  n_jobs=-1)

gs.fit(data,target_all)
print(gs.best_score_)
print(gs.best_params_)